In [226]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import simplefix
from simplefix import FixMessage, FixParser
import socket
import ssl
import os

#### Credentials Setup

In [237]:
# Setup credentials for connection to cTrader
socket_buffer = 1024
server_url = "h35.p.ctrader.com"
server_port = 5211

sender = "demo.ctrader.3988979"
target = "CSERVER"
username = "3988979"
password = "SamaTrades123!"

#### Connection Setup

Create dynamic message sequence numbers that reset on login

In [238]:
global seq_num

In [239]:
def ctrader_connection():
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    ssock = context.wrap_socket(sock, server_hostname=server_url)

    try:
        ssock.connect((server_url, server_port))
        print(f"Connected to {server_url} on port {server_port}")
        return ssock
    except socket.error as socket_err:
        print(f"Socket error occurred: {socket_err}")
        raise
        
# Create the socket connection
active_socket = ctrader_connection()

Connected to h35.p.ctrader.com on port 5211


In [243]:
def send_ctrader_request(socket, fix_message):
    parser = simplefix.FixParser()
    seq_num = 1
    if fix_message.get(35 == "A"):
        fix_message.append_pair(34, seq_num, header=True)
        
    else:
        seq_num += 1
        fix_message.append_pair(34, seq_num, header=True)
        
    encoded_message = fix_message.encode()
    
    print(f"Seq Num : {seq_num}")
        
    try:
        # Send the message bytes to the backend
        socket.send(encoded_message)
        # Fetch the API response
        response = socket.recv(socket_buffer)
        print("API response received:", response)
        # Use the parser to decode the API response
        parser.append_buffer(response)
        # Parse the message to return
        return_val = parser.get_message()
        return return_val
    except socket.error as socket_err:
        print(f"Socket error occurred: {socket_err}")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

#### SimpleFix Library Setup

In [244]:
def generate_login_request():
    # Create the FIX message
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, 126)
    message.append_pair(35, "A")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    #message.append_pair(34, seq_num, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(57, "QUOTE")
    message.append_pair(98, 0)
    message.append_pair(108, 0)
    # Reset all sequence connections
    message.append_pair(141, "Y")
    message.append_pair(553,username)
    message.append_pair(554,password)
    message.append_pair(10, 131)
    return message

def generate_logout_request():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, 126)
    message.append_pair(35, 5)
    message.append_pair(49, sender)
    message.append_pair(56, target)
    #message.append_pair(34, 1, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(57, "QUOTE")
    message.append_pair(98, 0)
    message.append_pair(108,0)
    message.append_pair(141, "Y")
    message.append_pair(553,username)
    message.append_pair(554,password)
    message.append_pair(10, 131)
    return message

def generate_market_request():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, 135)
    message.append_pair(35, "V")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    #message.append_pair(34, 2, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(57, "QUOTE")
    message.append_pair(262, "EURUSD")
    message.append_pair(263, 1)
    message.append_pair(264, 0)
    message.append_pair(265, 1)
    message.append_pair(267, 2)
    message.append_pair(269, 0)
    message.append_pair(269, 1)
    message.append_pair(146, 1)
    message.append_pair(55,1)
    message.append_pair(10, 94)
    return message


def submit_order():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(11, 8487874)
    message.append_pair(34, 12, header=True)
    message.append_pair(141, "Y")
    message.append_pair(55, 1)
    message.append_pair(54, 1)
    message.append_pair(57, "TRADE")
    message.append_pair(371, 57)
    message.append_pair(372, "D")
    message.append_pair(373, "TRADE")
    message.append_pair(35, "D")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_utc_timestamp(52)
    message.append_pair(38, 100)
    message.append_pair(40, 1)
    message.append_pair(10, 246)
    return message

#### Initiate a session to CTrader

In [245]:
login_response_message = send_ctrader_request(active_socket, generate_login_request())
if login_response_message.get(35).decode() == "A":
    print("Login Successful")
else:
    print(f"Login failed : {login_response_status.get(58)}")

Seq Num : 2
API response received: b'8=FIX.4.4\x019=96\x0135=A\x0134=1\x0149=CSERVER\x0150=QUOTE\x0152=20231211-17:57:18.051\x0156=demo.ctrader.3988979\x0198=0\x01108=0\x01141=Y\x0110=180\x01'
Seq Num : 2
API response received: b'8=FIX.4.4\x019=124\x0135=j\x0134=2\x0149=CSERVER\x0150=QUOTE\x0152=20231211-17:57:18.051\x0156=demo.ctrader.3988979\x0158=ALREADY_LOGGED_IN:Already logged in\x01380=0\x0110=053\x01'
8=FIX.4.4|9=124|35=j|34=2|49=CSERVER|50=QUOTE|52=20231211-17:57:18.051|56=demo.ctrader.3988979|58=ALREADY_LOGGED_IN:Already logged in|380=0|10=053


#### Request Market Data

In [ ]:
market_data_response = send_ctrader_request(active_socket, generate_market_request())
print(market_data_response)

#### Submit an order

In [ ]:
order_submission_response = send_ctrader_request(active_socket, submit_order())
print(order_submission_response)